In [1]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
!pip install gmaps
import gmaps
import os

# Import API key
from api_keys import g_key





In [2]:
weather_py = "../output_data/cities.csv"
cities = pd.read_csv(weather_py)
cities.head()
gmaps.configure(api_key = g_key)

In [3]:
cities=cities.rename(columns = {"Max Temp":"MaxTemp", "Wind Speed":"WindSpeed"})
cities["Humidity"].max()

100

In [4]:
min_temp= 21.111
max_temp = 26.666
wind_speed = 10
cloudiness = 0

In [5]:
good_cities= cities.query(f"MaxTemp < {max_temp} and MaxTemp > {min_temp} and WindSpeed < {wind_speed} and Cloudiness == {cloudiness}")
good_cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,MaxTemp,WindSpeed
3,3,esna,0,EG,1604018012,31,25.29,32.55,25.00,2.90
19,19,bosaso,0,SO,1604018014,78,11.28,49.18,25.74,0.72
26,26,busselton,0,AU,1604017917,31,-33.65,115.33,23.89,3.78
46,46,taoudenni,0,ML,1604018016,18,22.68,-3.98,24.70,6.92
105,105,airai,0,TL,1604017821,53,-8.93,125.41,24.13,0.93


In [6]:
hotel_df = good_cities
hotel_df['Hotel Name']=''
hotel_df.head()

<ipython-input-6-b545f72d5298>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=''


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,MaxTemp,WindSpeed,Hotel Name
3,3,esna,0,EG,1604018012,31,25.29,32.55,25.00,2.90,
19,19,bosaso,0,SO,1604018014,78,11.28,49.18,25.74,0.72,
26,26,busselton,0,AU,1604017917,31,-33.65,115.33,23.89,3.78,
46,46,taoudenni,0,ML,1604018016,18,22.68,-3.98,24.70,6.92,
105,105,airai,0,TL,1604017821,53,-8.93,125.41,24.13,0.93,


In [9]:
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

In [11]:
locations["Lat"]= locations["Lat"].astype(str)
locations["Lng"]=locations["Lng"].astype(str)

<ipython-input-11-e18913a566ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations["Lat"]= locations["Lat"].astype(str)
<ipython-input-11-e18913a566ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations["Lng"]=locations["Lng"].astype(str)


In [12]:
hotels = pd.DataFrame({"City": hotel_df["City"],
                      "Country": hotel_df["Country"]})
hotels["Hotel Name"]=''
hotels["Lat"]=''
hotels["Lng"]=''

In [18]:
for i,row in locations.iterrows():
    try:
        target_search="lodging"
        target_radius="5000"
        target_coordinates=row["lat"]+','+row["lng"]
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
            "location": target_coordinates,
            "keyword": target_search,
            "radius": target_radius,
            "key": gkey
            }

        response = requests.get(base_url, params=params).json()

        lat = response["results"][0]["geometry"]["location"]["lat"]
        lng = response["results"][0]["geometry"]["location"]["lng"]
        name= response["results"][0]["name"]


        hotels.loc[i,'Lat']=lat
        hotels.loc[i,'Lng']=lng
        hotels.loc[i,'Hotel Name']=name

        hotel_df.loc[i,'Hotel Name']=name
    except:
        hotels.loc[i,'Hotel Name']="Hotel not found!"
        hotel_df.loc[i,'Hotel Name']="Hotel not found!"

C:\Users\kaleb\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [19]:
hotel_df.dropna(inplace=True)
hotel_df['Date']=hotel_df['Date'].apply(lambda x: '%.0f' % x)
hotel_df.head(50)

<ipython-input-19-897ca9c1d9a6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.dropna(inplace=True)
<ipython-input-19-897ca9c1d9a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Date']=hotel_df['Date'].apply(lambda x: '%.0f' % x)


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,MaxTemp,WindSpeed,Hotel Name
3,3,esna,0,EG,1604018012,31,25.29,32.55,25.00,2.90,Hotel not found!
19,19,bosaso,0,SO,1604018014,78,11.28,49.18,25.74,0.72,Hotel not found!
26,26,busselton,0,AU,1604017917,31,-33.65,115.33,23.89,3.78,Hotel not found!
46,46,taoudenni,0,ML,1604018016,18,22.68,-3.98,24.70,6.92,Hotel not found!
105,105,airai,0,TL,1604017821,53,-8.93,125.41,24.13,0.93,Hotel not found!
137,137,lazaro cardenas,0,DO,1604018025,86,19.46,-71.33,25.56,1.34,Hotel not found!
190,190,sur,0,OM,1604017808,65,22.57,59.53,21.68,2.44,Hotel not found!
196,196,tari,0,NG,1604018033,25,12.41,4.49,26.06,2.87,Hotel not found!
241,241,dire,0,ML,1604018038,36,12.28,-10.97,22.04,0.70,Hotel not found!
253,253,tiebissou,0,CI,1604018039,100,7.16,-5.23,23.00,1.36,Hotel not found!


In [21]:
hotels.dropna(inplace=True)
hotels.head(50)

,City,Country,Hotel Name,Lat,Lng
3,esna,EG,Hotel not found!,,
19,bosaso,SO,Hotel not found!,,
26,busselton,AU,Hotel not found!,,
46,taoudenni,ML,Hotel not found!,,
105,airai,TL,Hotel not found!,,
137,lazaro cardenas,DO,Hotel not found!,,
190,sur,OM,Hotel not found!,,
196,tari,NG,Hotel not found!,,
241,dire,ML,Hotel not found!,,
253,tiebissou,CI,Hotel not found!,,


In [26]:
hotel_info = [info_box_template.format(**row) for index, row in hotels.iterrows()]
loca = hotel_df[["Lat", "Lng"]]
weights=good_cities['Humidity']
markers = gmaps.marker_layer(loca,info_box_content=hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))